In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import pandas as pd
from bsedata.bse import BSE 
import time
pd.set_option('display.max_rows',500)
pd.set_option('display.max_columns',500)
pd.set_option('display.width',1000)

In [4]:
github_excel="https://raw.githubusercontent.com/freudeg0/Stock-Market-Data-ETL-pipleine/main/Equity.xlsx"
engine='openpyxl'
EquityDF=pd.read_excel(github_excel,engine=engine)

In [5]:
EquityDF.head()

,Security Code,Issuer Name,Security Id,Security Name,Status,Group,Face Value,ISIN No,Industry,Instrument,Sector Name,Industry New Name,Igroup Name,ISubgroup Name
0,500002,ABB India Limited,ABB,ABB India Limited,Active,A,2.0,INE117A01022,Heavy Electrical Equipment,Equity,Industrials,Capital Goods,Electrical Equipment,Heavy Electrical Equipment
1,500003,Aegis Logistics Ltd.,AEGISLOG,AEGIS LOGISTICS LTD.,Active,A,1.0,INE208C01025,Trading - Gas,Equity,Energy,"Oil, Gas & Consumable Fuels",Gas,Trading - Gas
2,500008,Amara Raja Energy & Mobility Limited,ARE&M,Amara Raja Energy & Mobility Ltd,Active,A,1.0,INE885A01032,Auto Components & Equipments,Equity,Consumer Discretionary,Automobile and Auto Components,Auto Components,Auto Components & Equipments
3,500009,"Ambalal Sarabhai Enterprise Ltd.,",AMBALALSA,AMBALAL SARABHAI ENTERPRISES LTD.,Active,XT,10.0,INE432A01017,Pharmaceuticals,Equity,Healthcare,Healthcare,Pharmaceuticals & Biotechnology,Pharmaceuticals
4,500012,"Andhra Petrochemicals Ltd.,",ANDHRAPET,ANDHRA PETROCHEMICALS LTD.,Active,X,10.0,INE714B01016,Commodity Chemicals,Equity,Commodities,Chemicals,Chemicals & Petrochemicals,Commodity Chemicals


Calling BSE data

In [6]:
EquityDF.dtypes

Security Code          int64
Issuer Name           object
Security Id           object
Security Name         object
Status                object
Group                 object
Face Value           float64
ISIN No               object
Industry              object
Instrument            object
Sector Name           object
Industry New Name     object
Igroup Name           object
ISubgroup Name        object
dtype: object

In [7]:
list0f_Nift50_Stock_IDs=["ADANIENT", "ADANIPORTS", "APOLLOHOSP", "ASIANPAINT", "AXISBANK",
    "BAJAJ-AUTO", "BAJFINANCE", "BAJAJFINSV", "BPCL", "BHARTIARTL",
    "BRITANNIA", "CIPLA", "COALINDIA", "DIVISLAB", "DRREDDY", "EICHERMOT",
    "GRASIM", "HCLTECH", "HDFCBANK", "HDFCLIFE", "HEROMOTOCO", "HINDALCO",
    "HINDUNILVR", "ICICIBANK", "ITC", "INDUSINDBK", "INFY", "JSWSTEEL",
    "KOTAKBANK", "LTIM", "LT", "M&M", "MARUTI", "NTPC", "NESTLEIND",
    "ONGC", "POWERGRID", "RELIANCE", "SBILIFE", "SBIN", "SUNPHARMA",
    "TCS", "TATACONSUM", "TATAMOTORS", "TATASTEEL", "TECHM", "TITAN",
    "UPL", "ULTRACEMCO", "WIPRO"
]
EquityDF['Security Code'] = EquityDF['Security Code'].astype(str) #convert to string type
nifty50df=EquityDF[EquityDF['Security Id'].isin(list0f_Nift50_Stock_IDs)].reset_index(drop=True)
nifty50df.columns=nifty50df.columns.str.replace(' ','')

In [8]:
nifty50df.tail()

,SecurityCode,IssuerName,SecurityId,SecurityName,Status,Group,FaceValue,ISINNo,Industry,Instrument,SectorName,IndustryNewName,IgroupName,ISubgroupName
45,532978,Bajaj Finserv Limited,BAJAJFINSV,BAJAJ FINSERV LTD.,Active,A,1.0,INE918I01026,Holding Company,Equity,Financial Services,Financial Services,Finance,Holding Company
46,533278,Coal India Limited,COALINDIA,COAL INDIA LTD.,Active,A,10.0,INE522F01014,Coal,Equity,Energy,"Oil, Gas & Consumable Fuels",Consumable Fuels,Coal
47,540005,LTIMindtree Limited,LTIM,LTIMindtree Ltd,Active,A,1.0,INE214T01019,Computers - Software & Consulting,Equity,Information Technology,Information Technology,IT - Software,Computers - Software & Consulting
48,540719,SBI Life Insurance Company Limited,SBILIFE,SBI Life Insurance Company Ltd,Active,A,10.0,INE123W01016,Life Insurance,Equity,Financial Services,Financial Services,Insurance,Life Insurance
49,540777,HDFC LIFE INSURANCE COMPANY LIMITED,HDFCLIFE,HDFC Life Insurance Company Ltd,Active,A,10.0,INE795G01014,Life Insurance,Equity,Financial Services,Financial Services,Insurance,Life Insurance


In [9]:
nifty50df.dtypes

SecurityCode        object
IssuerName          object
SecurityId          object
SecurityName        object
Status              object
Group               object
FaceValue          float64
ISINNo              object
Industry            object
Instrument          object
SectorName          object
IndustryNewName     object
IgroupName          object
ISubgroupName       object
dtype: object

Extracting from API

In [10]:
bseObject = BSE(update_codes=True)  ## Bse Lib Object

listof_stockDicts = []
sqcode_ListNifty50 = nifty50df['SecurityCode'].values  
for sqCode in sqcode_ListNifty50:
    try:
        stockDict = bseObject.getQuote(sqCode) #key-value pair
        stockDict.pop("buy", None)
        stockDict.pop("sell", None)
        listof_stockDicts.append(stockDict)
        time.sleep(0.5)
    except IndexError:
        print(f"IndexError for {sqCode}: Data not available")

nifty50DailyTable = pd.DataFrame(listof_stockDicts)
nifty50DailyTable.head()


,companyName,currentValue,change,pChange,updatedOn,securityID,scripCode,group,faceValue,industry,previousClose,previousOpen,dayHigh,dayLow,52weekHigh,52weekLow,weightedAvgPrice,totalTradedValue,totalTradedQuantity,2WeekAvgQuantity,marketCapFull,marketCapFreeFloat
0,Bajaj Finance Limited,6601.80,-4.10,-0.06,06 Feb 24 | 04:00 PM,BAJFINANCE,500034,A / S&P BSE SENSEX,2.00,Financial Services,6605.90,6669.95,6697.00,6535.15,8190.00,5487.25,6608.10,30.49 Cr.,0.46 Lakh,0.90 Lakh,"4,08,006.47 Cr.","1,83,602.91 Cr."
1,CIPLA LTD.,1425.55,-7.90,-0.55,06 Feb 24 | 04:00 PM,CIPLA,500087,A / S&P BSE 100,2.00,Healthcare,1433.45,1446.55,1446.55,1413.80,1447.05,852.00,1425.49,3.43 Cr.,0.24 Lakh,1.92 Lakh,"1,15,093.14 Cr.","75,961.47 Cr."
2,STATE BANK OF INDIA,650.90,7.80,1.21,06 Feb 24 | 04:00 PM,SBIN,500112,A / S&P BSE SENSEX,1.00,Financial Services,643.10,644.35,654.95,637.80,660.40,501.85,644.70,52.38 Cr.,8.12 Lakh,12.55 Lakh,"5,80,902.99 Cr.","2,49,788.29 Cr."
3,Titan Company Limited,3559.75,7.15,0.20,06 Feb 24 | 04:00 PM,TITAN,500114,A / S&P BSE SENSEX,1.00,Consumer Durables,3552.60,3552.60,3578.50,3532.40,3885.00,2320.00,3549.74,11.24 Cr.,0.32 Lakh,0.65 Lakh,"3,16,029.68 Cr.","1,48,533.95 Cr."
4,DR.REDDY'S LABORATORIES LTD.,6165.55,73.75,1.21,06 Feb 24 | 04:00 PM,DRREDDY,500124,A / S&P BSE 100,5.00,Healthcare,6091.80,6080.00,6191.50,6078.00,6191.50,4288.55,6167.44,5.48 Cr.,0.09 Lakh,0.19 Lakh,"1,02,842.35 Cr.","75,074.92 Cr."


Data Transformation

In [11]:
nifty50DailyTable.rename(columns={'group': 'sharegroup'}, inplace=True)
nifty50DailyTable.rename(columns={'52weekHigh': 'fiftytwoweekHigh'}, inplace=True)
nifty50DailyTable.rename(columns={'52weekLow': 'fiftytwoweekLow'}, inplace=True)
nifty50DailyTable.rename(columns={'2WeekAvgQuantity': 'twoWeekAvgQuantity'}, inplace=True)
nifty50DailyTable.rename(columns={'change': 'change_value'}, inplace=True)

In [12]:
nifty50DailyTable['updatedOn'] = pd.to_datetime(nifty50DailyTable['updatedOn'], format='%d %b %y | %I:%M %p', errors='coerce')

In [13]:
nifty50DailyTable.dtypes

companyName                    object
currentValue                   object
change_value                   object
pChange                        object
updatedOn              datetime64[ns]
securityID                     object
scripCode                      object
sharegroup                     object
faceValue                      object
industry                       object
previousClose                  object
previousOpen                   object
dayHigh                        object
dayLow                         object
fiftytwoweekHigh               object
fiftytwoweekLow                object
weightedAvgPrice               object
totalTradedValue               object
totalTradedQuantity            object
twoWeekAvgQuantity             object
marketCapFull                  object
marketCapFreeFloat             object
dtype: object

In [14]:
nifty50DailyTable.isnull().sum()

companyName            0
currentValue           0
change_value           0
pChange                0
updatedOn              0
securityID             0
scripCode              0
sharegroup             0
faceValue              0
industry               0
previousClose          0
previousOpen           0
dayHigh                0
dayLow                 0
fiftytwoweekHigh       0
fiftytwoweekLow        0
weightedAvgPrice       0
totalTradedValue       0
totalTradedQuantity    0
twoWeekAvgQuantity     0
marketCapFull          0
marketCapFreeFloat     0
dtype: int64

In [15]:
if pd.isna(nifty50DailyTable['updatedOn']).any():
    print("There are invalid or missing date values in the 'updatedOn' column.")
else:
    # Extract date from 'updatedOn' column and convert the column to datetime
    nifty50DailyTable['updatedOn'] = pd.to_datetime(nifty50DailyTable['updatedOn'].dt.date)

In [16]:
if 'totalTradedValueCr' not in nifty50DailyTable.columns:
    nifty50DailyTable['totalTradedValueCr'] = pd.to_numeric(nifty50DailyTable['totalTradedValue'].str.replace(',', '').str.replace(' Cr.', '', regex=True), errors='coerce')  # Convert to numeric and handle 'Cr.'
    nifty50DailyTable['totalTradedQuantityLakh'] = pd.to_numeric(nifty50DailyTable['totalTradedQuantity'].str.replace(',', '').str.replace(' Lakh', '', regex=True), errors='coerce')  # Convert to numeric and handle 'Lakh'
    nifty50DailyTable['twoWeekAvgQuantityLakh'] = pd.to_numeric(nifty50DailyTable['twoWeekAvgQuantity'].str.replace(',', '').str.replace(' Lakh', '', regex=True), errors='coerce')  # Convert to numeric and handle 'Lakh'
    nifty50DailyTable['marketCapFullCr'] = pd.to_numeric(nifty50DailyTable['marketCapFull'].str.replace(',', '').str.replace(' Cr.', '', regex=True), errors='coerce')  # Convert to numeric and handle 'Cr.'
    nifty50DailyTable['marketCapFreeFloatCr'] = pd.to_numeric(nifty50DailyTable['marketCapFreeFloat'].str.replace(',', '').str.replace(' Cr.', '', regex=True), errors='coerce')  # Convert to numeric and handle 'Cr.'


In [17]:
nifty50DailyTable.dtypes

companyName                        object
currentValue                       object
change_value                       object
pChange                            object
updatedOn                  datetime64[ns]
securityID                         object
scripCode                          object
sharegroup                         object
faceValue                          object
industry                           object
previousClose                      object
previousOpen                       object
dayHigh                            object
dayLow                             object
fiftytwoweekHigh                   object
fiftytwoweekLow                    object
weightedAvgPrice                   object
totalTradedValue                   object
totalTradedQuantity                object
twoWeekAvgQuantity                 object
marketCapFull                      object
marketCapFreeFloat                 object
totalTradedValueCr                float64
totalTradedQuantityLakh           

In [18]:
nifty50DailyTable.drop(['totalTradedValue', 'totalTradedQuantity','twoWeekAvgQuantity', 'marketCapFull', 'marketCapFreeFloat'], axis=1,inplace=True)
nifty50DailyTable.head()

,companyName,currentValue,change_value,pChange,updatedOn,securityID,scripCode,sharegroup,faceValue,industry,previousClose,previousOpen,dayHigh,dayLow,fiftytwoweekHigh,fiftytwoweekLow,weightedAvgPrice,totalTradedValueCr,totalTradedQuantityLakh,twoWeekAvgQuantityLakh,marketCapFullCr,marketCapFreeFloatCr
0,Bajaj Finance Limited,6601.80,-4.10,-0.06,2024-02-06,BAJFINANCE,500034,A / S&P BSE SENSEX,2.00,Financial Services,6605.90,6669.95,6697.00,6535.15,8190.00,5487.25,6608.10,30.49,0.46,0.90,408006.47,183602.91
1,CIPLA LTD.,1425.55,-7.90,-0.55,2024-02-06,CIPLA,500087,A / S&P BSE 100,2.00,Healthcare,1433.45,1446.55,1446.55,1413.80,1447.05,852.00,1425.49,3.43,0.24,1.92,115093.14,75961.47
2,STATE BANK OF INDIA,650.90,7.80,1.21,2024-02-06,SBIN,500112,A / S&P BSE SENSEX,1.00,Financial Services,643.10,644.35,654.95,637.80,660.40,501.85,644.70,52.38,8.12,12.55,580902.99,249788.29
3,Titan Company Limited,3559.75,7.15,0.20,2024-02-06,TITAN,500114,A / S&P BSE SENSEX,1.00,Consumer Durables,3552.60,3552.60,3578.50,3532.40,3885.00,2320.00,3549.74,11.24,0.32,0.65,316029.68,148533.95
4,DR.REDDY'S LABORATORIES LTD.,6165.55,73.75,1.21,2024-02-06,DRREDDY,500124,A / S&P BSE 100,5.00,Healthcare,6091.80,6080.00,6191.50,6078.00,6191.50,4288.55,6167.44,5.48,0.09,0.19,102842.35,75074.92


Connect to MySQL database and load the data

In [19]:
import mysql.connector
from sqlalchemy import create_engine, inspect
from sqlalchemy.exc import SQLAlchemyError
import sqlalchemy as sa

host = 'localhost'
database = 'nifty50'
username = 'root'
password = 'dontaskmypass'

# MySQL connection string
conn_str = f'mysql+mysqlconnector://{username}:{password}@{host}/{database}'

# Create an SQLAlchemy engine
engine = create_engine(conn_str)

def create_connection(conn_str):
    conn = mysql.connector.connect(host=host, database=database, user=username, password=password)
    cursor = conn.cursor()
    return conn, cursor

try:
    # Try to connect to MySQL using the engine
    connection = engine.connect()
    print("Connection successful!")
    connection.close()
    conn, cursor = create_connection(conn_str)
    inspector = inspect(engine)

    nifty50_table_name = 'nifty50_dailydata'
    if not inspector.has_table(nifty50_table_name):
        nifty50_table_schema = f'''
        CREATE TABLE {nifty50_table_name} (
            companyName VARCHAR(255),
            currentValue FLOAT,
            change_value FLOAT,  
            pChange FLOAT,
            updatedOn DATE,
            securityID VARCHAR(255),
            scripCode VARCHAR(255),
            sharegroup VARCHAR(255),
            faceValue FLOAT,
            industry VARCHAR(255),
            previousClose FLOAT,
            previousOpen FLOAT,
            dayHigh FLOAT,
            dayLow FLOAT,
            fiftytwoweekHigh FLOAT,
            fiftytwoweekLow FLOAT,
            weightedAvgPrice FLOAT,
            totalTradedQuantityLakh FLOAT,
            totalTradedValueCr FLOAT,
            twoWeekAvgQuantityLakh FLOAT,
            marketCapFullCr FLOAT,
            marketCapFreeFloatCr FLOAT
        );
        '''
        # Execute the schema to create the table
        cursor.execute(nifty50_table_schema)
        conn.commit()
        conn.close()

    with engine.begin() as engineConn:
        sql_max_updatedOn = pd.read_sql_query(sa.text(f'SELECT MAX(updatedOn) FROM {nifty50_table_name}'), engineConn).iloc[0, 0]
        print(sql_max_updatedOn)
        # Assuming nifty50DailyTable is a DataFrame with a column 'updatedOn'
        df_max_updatedOn = nifty50DailyTable['updatedOn'].max()
        print(df_max_updatedOn)
        if (pd.isnull(sql_max_updatedOn)) and (not pd.isnull(df_max_updatedOn)):
            nifty50DailyTable.to_sql(nifty50_table_name, engine, index=False, if_exists='append', method='multi')
            print("Daily Data didn't exist, but now inserted successfully.")
        else:
            if (df_max_updatedOn > pd.Timestamp(sql_max_updatedOn)):
                nifty50DailyTable.to_sql(nifty50_table_name, engine, index=False, if_exists='append', method='multi')
                print("Data appended successfully.")
            else:
                print("No new data to append.")

    company_table_name = 'nifty50_companydata'
    if not inspector.has_table(company_table_name):
        # Define the table schema based on the 'Equity' DataFrame columns
        company_table_schema = f'''
        CREATE TABLE {company_table_name} (
            securityCode VARCHAR(255),
            issuerName VARCHAR(255),
            securityId VARCHAR(255),
            securityName VARCHAR(255),
            status VARCHAR(255),
            CompanyGroup VARCHAR(255),
            faceValue FLOAT,
            isinNo VARCHAR(255),
            industry VARCHAR(255),
            instrument VARCHAR(255),
            sectorName VARCHAR(255),
            industryNewName VARCHAR(255),
            igroupName VARCHAR(255),
            iSubgroupName VARCHAR(255)
        );
        '''

        # Execute the schema to create the 'company' table
        conn, cursor = create_connection(conn_str)
        cursor.execute(company_table_schema)
        # Commit the changes and close the connection
        conn.commit()
        conn.close()
        nifty50df.rename(columns={'Group': 'CompanyGroup'}, inplace=True)
        nifty50df.to_sql(company_table_name, engine, index=False, if_exists='append', method='multi')
    else:
        print("company Table already exists, hence skipping")

except SQLAlchemyError as e:
    print(f"Error connecting to MySQL Server: {e}")

Connection successful!
2024-02-05
2024-02-06 00:00:00
Data appended successfully.
company Table already exists, hence skipping
